# Brain Tumor Classification with VGG16

This notebook demonstrates an end-to-end workflow for brain tumor classification using VGG16 transfer learning models. It includes data preprocessing, multiple model variants, training, evaluation, and comprehensive result generation.

## Overview

1. Mount Google Drive and set up the environment
2. Update the repository
3. Install dependencies
4. Set up paths to the dataset and results directories
5. Load and explore the dataset
6. Preprocess the data with augmentation
7. Build multiple VGG16 model variants
8. Train and validate all models
9. Evaluate performance with comprehensive metrics
10. Generate visualizations and save all required outputs

## 1. Mount Google Drive and Setup Environment

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Verify TensorFlow and GPU availability
import tensorflow as tf
import platform

print('TensorFlow version:', tf.__version__)
print('Python version:', platform.python_version())
print('GPUs available:', tf.config.list_physical_devices('GPU'))

# Set seed for reproducibility
import numpy as np
import random
import os

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## 2. Update Repository and Install Dependencies

In [ ]:
# Navigate to repository directory and pull updates
%cd /content/drive/MyDrive/SE4050-Deep-Learning-Assignment
!git pull origin main

# Install required packages
!pip install -q opencv-python
!pip install -q scikit-learn
!pip install -q matplotlib
!pip install -q seaborn

print("✅ Repository updated and dependencies installed")

## 3. Import Required Libraries

In [ ]:
# Import all required libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import json
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
import sys
sys.path.append('/content/drive/MyDrive/SE4050-Deep-Learning-Assignment')

from src.common.dataset_utils import create_datasets
from src.common.preprocessing import get_augmentation_pipeline, verify_dataset, split_and_copy
from src.common.gradcam import generate_gradcam

print("✅ All libraries imported successfully")

## 4. Setup Paths and Configuration

In [ ]:
# Configuration
BASE_DIR = "/content/drive/MyDrive"
PROJECT_DIR = os.path.join(BASE_DIR, "SE4050-Deep-Learning-Assignment")
RAW_DATA_DIR = os.path.join(BASE_DIR, "BrainTumor")
PROCESSED_DATA_DIR = os.path.join(BASE_DIR, "BrainTumor", "data", "processed")
RESULTS_DIR = os.path.join(BASE_DIR, "BrainTumor", "Result", "vgg16")

# Model configuration
INPUT_SHAPE = (224, 224, 3)  # VGG16 optimal input size
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.0001

# Create necessary directories
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "gradcam"), exist_ok=True)

print(f"✅ Paths configured:")
print(f"   Raw data: {RAW_DATA_DIR}")
print(f"   Processed data: {PROCESSED_DATA_DIR}")
print(f"   Results: {RESULTS_DIR}")
print(f"   Input shape: {INPUT_SHAPE}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Epochs: {EPOCHS}")

## 5. Data Preprocessing

Check if preprocessing is needed and perform data preprocessing if required.

In [ ]:
# Check if processed data exists
train_dir = os.path.join(PROCESSED_DATA_DIR, "train")
val_dir = os.path.join(PROCESSED_DATA_DIR, "val")
test_dir = os.path.join(PROCESSED_DATA_DIR, "test")

processed_exists, class_folders_valid = verify_dataset(PROCESSED_DATA_DIR)

if processed_exists and class_folders_valid:
    print("✅ Processed data already exists with valid class folders. Skipping preprocessing.")
else:
    print("🔄 Starting preprocessing...")
    print(f"Reading raw images from: {RAW_DATA_DIR}")
    print(f"Saving processed images to: {PROCESSED_DATA_DIR}")

    # Check if raw data exists
    yes_dir = os.path.join(RAW_DATA_DIR, "yes")
    no_dir = os.path.join(RAW_DATA_DIR, "no")
    
    if os.path.exists(RAW_DATA_DIR) and os.path.exists(yes_dir) and os.path.exists(no_dir):
        total_files = split_and_copy(RAW_DATA_DIR, PROCESSED_DATA_DIR, ["yes", "no"])
        print(f"✅ Preprocessing completed successfully! Processed {total_files} images.")
    else:
        print("⚠️  Could not find expected yes/no folders in the raw data directory.")
        print("Please make sure your Google Drive contains the correct folder structure.")

## 6. Load and Explore Dataset

In [ ]:
# Create datasets with data augmentation for VGG16 (224x224 input)
augment = get_augmentation_pipeline()
train_ds, val_ds, test_ds, class_names = create_datasets(
    PROCESSED_DATA_DIR, 
    BATCH_SIZE, 
    img_size=(224, 224),  # VGG16 optimal size
    augment_fn=augment
)

print(f"✅ Dataset loaded successfully!")
print(f"   Class names: {class_names}")
print(f"   Input shape: {INPUT_SHAPE}")

# Display sample images
plt.figure(figsize=(12, 8))
for images, labels in train_ds.take(1):
    for i in range(min(9, len(images))):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"Class: {class_names[int(labels[i])]}")
        plt.axis('off')
plt.suptitle("Sample Images from Training Dataset", fontsize=16)
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "sample_images.png"), dpi=300, bbox_inches='tight')
plt.show()

# Dataset statistics
train_samples = sum(1 for _ in train_ds.unbatch())
val_samples = sum(1 for _ in val_ds.unbatch())
test_samples = sum(1 for _ in test_ds.unbatch())

print(f"\n📊 Dataset Statistics:")
print(f"   Training samples: {train_samples}")
print(f"   Validation samples: {val_samples}")
print(f"   Test samples: {test_samples}")
print(f"   Total samples: {train_samples + val_samples + test_samples}")

## 7. Build VGG16 Model Variants

Create multiple VGG16 model variants for comparison:

In [ ]:
def build_vgg16_basic(input_shape=INPUT_SHAPE):
    """
    Basic VGG16 transfer learning model - frozen base
    """
    base_model = VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )
    base_model.trainable = False  # Freeze base layers
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

def build_vgg16_fine_tuned(input_shape=INPUT_SHAPE):
    """
    Fine-tuned VGG16 model - unfreeze last few layers
    """
    base_model = VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )
    
    # Freeze early layers, unfreeze last block
    base_model.trainable = True
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE/10),  # Lower LR for fine-tuning
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

def build_vgg16_enhanced(input_shape=INPUT_SHAPE):
    """
    Enhanced VGG16 model with additional regularization
    """
    base_model = VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )
    base_model.trainable = False
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.6),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# Build all model variants
models_dict = {
    'VGG16_Basic': build_vgg16_basic(),
    'VGG16_Fine_Tuned': build_vgg16_fine_tuned(),
    'VGG16_Enhanced': build_vgg16_enhanced()
}

# Display model summaries
for name, model in models_dict.items():
    print(f"\n{'='*50}")
    print(f"📋 {name} Model Summary")
    print(f"{'='*50}")
    model.summary()
    
    # Save model summary
    with open(os.path.join(RESULTS_DIR, f"{name}_summary.txt"), "w") as f:
        model.summary(print_fn=lambda x: f.write(x + "\n"))

print(f"\n✅ Created {len(models_dict)} model variants")

## 8. Model Training and Validation

Train all model variants and track their performance:

In [ ]:
# Training function
def train_model(model, name, epochs=EPOCHS):
    """Train a model with callbacks and return history"""
    
    # Create model-specific directory
    model_dir = os.path.join(RESULTS_DIR, name)
    os.makedirs(model_dir, exist_ok=True)
    
    # Callbacks
    callbacks = [
        ModelCheckpoint(
            filepath=os.path.join(model_dir, f"{name}_best.h5"),
            monitor="val_accuracy",
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        EarlyStopping(
            monitor="val_loss", 
            patience=5, 
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7,
            verbose=1
        )
    ]
    
    print(f"\n🚀 Training {name}...")
    print(f"   Epochs: {epochs}")
    print(f"   Model directory: {model_dir}")
    
    # Train the model
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    
    print(f"✅ {name} training completed!")
    return history

# Train all models
histories = {}
for name, model in models_dict.items():
    histories[name] = train_model(model, name)
    print(f"\n{'-'*50}\n")

print("🎉 All models trained successfully!")

## 9. Generate Training Plots

Create and save training history plots for all models:

In [ ]:
# Plot training history for all models
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('VGG16 Model Training Comparison', fontsize=16, fontweight='bold')

colors = ['blue', 'red', 'green']
model_names = list(histories.keys())

# Accuracy plots
for i, (name, history) in enumerate(histories.items()):
    # Training and validation accuracy
    axes[0, i].plot(history.history['accuracy'], label='Training', color=colors[i], linewidth=2)
    axes[0, i].plot(history.history['val_accuracy'], label='Validation', color=colors[i], linestyle='--', linewidth=2)
    axes[0, i].set_title(f'{name} - Accuracy', fontweight='bold')
    axes[0, i].set_xlabel('Epoch')
    axes[0, i].set_ylabel('Accuracy')
    axes[0, i].legend()
    axes[0, i].grid(True, alpha=0.3)
    
    # Training and validation loss
    axes[1, i].plot(history.history['loss'], label='Training', color=colors[i], linewidth=2)
    axes[1, i].plot(history.history['val_loss'], label='Validation', color=colors[i], linestyle='--', linewidth=2)
    axes[1, i].set_title(f'{name} - Loss', fontweight='bold')
    axes[1, i].set_xlabel('Epoch')
    axes[1, i].set_ylabel('Loss')
    axes[1, i].legend()
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "training_plots_comparison.png"), dpi=300, bbox_inches='tight')
plt.show()

# Create individual training plots for each model
for name, history in histories.items():
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Accuracy plot
    ax1.plot(history.history['accuracy'], label='Training', linewidth=2)
    ax1.plot(history.history['val_accuracy'], label='Validation', linewidth=2)
    ax1.set_title(f'{name} - Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Loss plot
    ax2.plot(history.history['loss'], label='Training', linewidth=2)
    ax2.plot(history.history['val_loss'], label='Validation', linewidth=2)
    ax2.set_title(f'{name} - Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle(f'{name} Training History', fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    # Save individual plot
    model_dir = os.path.join(RESULTS_DIR, name)
    plt.savefig(os.path.join(model_dir, f"{name}_training_plot.png"), dpi=300, bbox_inches='tight')
    plt.show()

print("✅ Training plots generated and saved!")

## 10. Model Evaluation and Metrics

Evaluate all models and identify the best performer:

In [ ]:
# Evaluate all models on test set
def evaluate_model(model, name):
    """Evaluate model on test set and return metrics"""
    print(f"\n📊 Evaluating {name}...")
    
    # Predictions
    y_true = []
    y_pred = []
    y_pred_proba = []
    
    for images, labels in test_ds:
        predictions = model.predict(images, verbose=0)
        y_true.extend(labels.numpy())
        y_pred_proba.extend(predictions.flatten())
        y_pred.extend((predictions > 0.5).astype(int).flatten())
    
    # Calculate metrics
    test_accuracy = accuracy_score(y_true, y_pred)
    test_precision = precision_score(y_true, y_pred)
    test_recall = recall_score(y_true, y_pred)
    test_f1 = f1_score(y_true, y_pred)
    
    # Get training metrics
    history = histories[name]
    train_acc = max(history.history['accuracy'])
    val_acc = max(history.history['val_accuracy'])
    train_loss = min(history.history['loss'])
    val_loss = min(history.history['val_loss'])
    
    metrics = {
        'model_name': name,
        'train_accuracy': float(train_acc),
        'val_accuracy': float(val_acc),
        'test_accuracy': float(test_accuracy),
        'train_loss': float(train_loss),
        'val_loss': float(val_loss),
        'test_precision': float(test_precision),
        'test_recall': float(test_recall),
        'test_f1_score': float(test_f1),
        'epochs_trained': len(history.history['accuracy'])
    }
    
    print(f"   Test Accuracy: {test_accuracy:.4f}")
    print(f"   Test Precision: {test_precision:.4f}")
    print(f"   Test Recall: {test_recall:.4f}")
    print(f"   Test F1-Score: {test_f1:.4f}")
    
    return metrics, y_true, y_pred, y_pred_proba

# Evaluate all models
all_metrics = {}
all_predictions = {}

for name, model in models_dict.items():
    # Load best model
    model_path = os.path.join(RESULTS_DIR, name, f"{name}_best.h5")
    if os.path.exists(model_path):
        model.load_weights(model_path)
        print(f"✅ Loaded best weights for {name}")
    
    metrics, y_true, y_pred, y_pred_proba = evaluate_model(model, name)
    all_metrics[name] = metrics
    all_predictions[name] = {'y_true': y_true, 'y_pred': y_pred, 'y_pred_proba': y_pred_proba}
    
    # Save individual model metrics
    model_dir = os.path.join(RESULTS_DIR, name)
    with open(os.path.join(model_dir, f"{name}_metrics.json"), "w") as f:
        json.dump(metrics, f, indent=4)

# Create comparison DataFrame
comparison_df = pd.DataFrame(all_metrics).T
print(f"\n📈 Model Comparison Summary:")
print("="*80)
print(comparison_df[['test_accuracy', 'test_precision', 'test_recall', 'test_f1_score']].round(4))

# Find best model based on test accuracy
best_model_name = comparison_df['test_accuracy'].idxmax()
best_model = models_dict[best_model_name]
best_metrics = all_metrics[best_model_name]

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Test Accuracy: {best_metrics['test_accuracy']:.4f}")
print(f"   Test F1-Score: {best_metrics['test_f1_score']:.4f}")

# Save comparison results
comparison_df.to_csv(os.path.join(RESULTS_DIR, "model_comparison.csv"))
with open(os.path.join(RESULTS_DIR, "all_metrics.json"), "w") as f:
    json.dump(all_metrics, f, indent=4)

## 11. Create Confusion Matrix

Generate confusion matrix for the best model:

In [ ]:
# Create confusion matrices for all models
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle('Confusion Matrices Comparison', fontsize=16, fontweight='bold')

for i, (name, predictions) in enumerate(all_predictions.items()):
    y_true = predictions['y_true']
    y_pred = predictions['y_pred']
    
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(ax=axes[i], cmap='Blues', values_format='d')
    axes[i].set_title(f'{name}', fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "confusion_matrices_comparison.png"), dpi=300, bbox_inches='tight')
plt.show()

# Create detailed confusion matrix for best model
best_predictions = all_predictions[best_model_name]
y_true_best = best_predictions['y_true']
y_pred_best = best_predictions['y_pred']

plt.figure(figsize=(8, 6))
cm_best = confusion_matrix(y_true_best, y_pred_best)
disp_best = ConfusionMatrixDisplay(confusion_matrix=cm_best, display_labels=class_names)
disp_best.plot(cmap='Blues', values_format='d')
plt.title(f'Confusion Matrix - {best_model_name} (Best Model)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "confusion_matrix.png"), dpi=300, bbox_inches='tight')
plt.show()

# Generate classification report for best model
report_best = classification_report(y_true_best, y_pred_best, target_names=class_names)
print(f"\n📋 Classification Report - {best_model_name}:")
print("="*60)
print(report_best)

# Save classification report
with open(os.path.join(RESULTS_DIR, "classification_report.txt"), "w") as f:
    f.write(f"Classification Report - {best_model_name}\n")
    f.write("="*60 + "\n")
    f.write(report_best)

print("✅ Confusion matrices and classification reports generated!")

## 12. Generate Grad-CAM Visualizations

Create Grad-CAM visualizations to understand what the best model focuses on:

In [ ]:
# Generate Grad-CAM visualizations for the best model
print(f"🔍 Generating Grad-CAM visualizations for {best_model_name}...")

# Create gradcam directory
gradcam_dir = os.path.join(RESULTS_DIR, "gradcam")
os.makedirs(gradcam_dir, exist_ok=True)

# Generate Grad-CAM for best model
try:
    generate_gradcam(
        model=best_model, 
        dataset=test_ds, 
        save_dir=gradcam_dir, 
        class_names=class_names,
        num_images=5,  # Generate for 5 images
        max_samples=3   # Limit to first 3 batches for speed
    )
    print("✅ Grad-CAM visualizations generated successfully!")
except Exception as e:
    print(f"⚠️  Grad-CAM generation failed: {str(e)}")
    print("Continuing without Grad-CAM...")

# Create a summary visualization showing sample predictions with confidence
plt.figure(figsize=(15, 10))
sample_count = 0
max_samples = 12

for images, labels in test_ds.take(2):  # Take 2 batches
    predictions = best_model.predict(images, verbose=0)
    
    for i in range(min(len(images), max_samples - sample_count)):
        if sample_count >= max_samples:
            break
            
        plt.subplot(3, 4, sample_count + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        true_label = class_names[int(labels[i])]
        pred_prob = predictions[i][0]
        pred_label = class_names[1] if pred_prob > 0.5 else class_names[0]
        confidence = pred_prob if pred_prob > 0.5 else (1 - pred_prob)
        
        # Color based on correctness
        color = 'green' if true_label == pred_label else 'red'
        
        plt.title(f'True: {true_label}\\nPred: {pred_label}\\nConf: {confidence:.3f}', 
                 color=color, fontsize=10)
        plt.axis('off')
        sample_count += 1
    
    if sample_count >= max_samples:
        break

plt.suptitle(f'{best_model_name} - Sample Predictions', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "sample_predictions.png"), dpi=300, bbox_inches='tight')
plt.show()

print("✅ Sample predictions visualization created!")

## 13. Save Best Model and Final Results

Save the best model and generate all required output files:

In [ ]:
# Save the best model as best_model.h5
best_model_path = os.path.join(RESULTS_DIR, "best_model.h5")
best_model.save(best_model_path)
print(f"✅ Best model saved: {best_model_path}")

# Save best model's training plot as training_plot.png
best_history = histories[best_model_name]
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(best_history.history['accuracy'], label='Training', linewidth=2)
plt.plot(best_history.history['val_accuracy'], label='Validation', linewidth=2)
plt.title(f'{best_model_name} - Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(best_history.history['loss'], label='Training', linewidth=2)
plt.plot(best_history.history['val_loss'], label='Validation', linewidth=2)
plt.title(f'{best_model_name} - Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.suptitle(f'Best Model Training History - {best_model_name}', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "training_plot.png"), dpi=300, bbox_inches='tight')
plt.show()

# Save metrics.json with best model metrics
final_metrics = {
    "best_model": best_model_name,
    "model_metrics": best_metrics,
    "training_config": {
        "input_shape": INPUT_SHAPE,
        "batch_size": BATCH_SIZE,
        "epochs": EPOCHS,
        "learning_rate": LEARNING_RATE
    },
    "dataset_info": {
        "class_names": class_names,
        "train_samples": train_samples,
        "val_samples": val_samples,
        "test_samples": test_samples
    },
    "all_models_comparison": all_metrics
}

with open(os.path.join(RESULTS_DIR, "metrics.json"), "w") as f:
    json.dump(final_metrics, f, indent=4)

print(f"✅ Final metrics saved: {os.path.join(RESULTS_DIR, 'metrics.json')}")

# Generate summary report
summary_report = f"""
# VGG16 Brain Tumor Classification - Results Summary

## Best Model: {best_model_name}
- **Test Accuracy**: {best_metrics['test_accuracy']:.4f}
- **Test Precision**: {best_metrics['test_precision']:.4f}
- **Test Recall**: {best_metrics['test_recall']:.4f}
- **Test F1-Score**: {best_metrics['test_f1_score']:.4f}

## Model Comparison
"""

for name, metrics in all_metrics.items():
    summary_report += f"\\n### {name}\\n"
    summary_report += f"- Test Accuracy: {metrics['test_accuracy']:.4f}\\n"
    summary_report += f"- Test F1-Score: {metrics['test_f1_score']:.4f}\\n"

summary_report += f"""

## Generated Files
- `best_model.h5` - Best performing model
- `training_plot.png` - Training history plots
- `confusion_matrix.png` - Confusion matrix
- `metrics.json` - Comprehensive metrics
- `gradcam/` - Grad-CAM visualizations
- `classification_report.txt` - Detailed classification report

## Configuration
- Input Shape: {INPUT_SHAPE}
- Batch Size: {BATCH_SIZE}
- Epochs: {EPOCHS}
- Learning Rate: {LEARNING_RATE}
"""

with open(os.path.join(RESULTS_DIR, "summary_report.md"), "w") as f:
    f.write(summary_report)

print("✅ Summary report generated!")

# List all generated files
print(f"\\n📁 Generated files in {RESULTS_DIR}:")
for file in sorted(os.listdir(RESULTS_DIR)):
    file_path = os.path.join(RESULTS_DIR, file)
    if os.path.isfile(file_path):
        print(f"   📄 {file}")
    elif os.path.isdir(file_path):
        print(f"   📁 {file}/")
        for subfile in sorted(os.listdir(file_path)):
            print(f"      📄 {subfile}")

print(f"\\n🎉 VGG16 Brain Tumor Classification completed successfully!")
print(f"📊 Best Model: {best_model_name} with {best_metrics['test_accuracy']:.4f} test accuracy")
print(f"📁 All results saved in: {RESULTS_DIR}")